# AstraZeneca - Lists
Aus irgend einem Grund erkennt Tabula die Daten nicht als eine Tabelle. Daher werden sie separat (multiple_tables=True) eingelesen und danach wieder zusammengeführt

## Info
(Auf der summierten Liste sind die Werte in HCP und HCO falsch. Gesamttotal stimmt, aber einzelne Kategorien sind falsch. Habs durchgerechnet, unseres stimmt) noch nicht überprüft in 2019

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("Switzerland_EFPIA Standard Disclosure Summary Report_2019_engl.pdf", pages='all', lattice=True, multiple_tables=True)

## Concat Tables
In der letzten Tabelle sind die Columns verschoben. Manuell zurücksetzen und zu einem DF joinen

In [3]:
df_last = df_list[len(df_list) - 1]

#Remove Columns 8 and 12
df_last = df_last.drop(columns=[8, 12])

#Reset Columns name
df_last.columns = [ i for i in range(0,13)]

df_list[len(df_list) - 1] = df_last

# Concat tables
df = pd.concat(df_list)

## Format Table

In [4]:
#Rename Columns
df_export = df.copy()
df_export.columns = ['empty1', 'name', 'location', 'country', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty2']

#Remove empty rows
df_export = remove_empty_columns(df_export)

#Remove all which have NaN in total
df_export = df_export[~df_export['total'].isnull()]

#Remove 2xspace
df_export = df_export.replace('    ', ' ', regex=True)

#Set Type
df_export = set_type_by_alphabetical_order(df_export)

#Basic String Conversions
df_export = basic_string_conversion(df_export)

#Amounts to Number
df_export = amounts_to_number(df_export)

#Remove Carination
df_export = remove_carination(df_export)

#Add empty fields
df_export = add_plz(df_export)
df_export = add_uci(df_export)

#Export
export_list(df_export, 'astrazeneca')

Duplicates found. Please check for duplicates: df_export[df_export.duplicated()]
saved


In [5]:
#write_to_excel(df_export, 'astrazeneca')
#write_to_csv(df_export, 'tmp.xlsx', True)